## 설치

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 7.3 MB/s 
     |████████████████████████████████| 163 kB 79.2 MB/s 
     |████████████████████████████████| 7.6 MB 56.2 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 7.5 MB/s 
     |████████████████████████████████| 212 kB 87.7 MB/s 
     |████████████████████████████████| 115 kB 86.7 MB/s 
     |████████████████████████████████| 127 kB 81.8 MB/s 
     |████████████████████████████████| 115 kB 84.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 8.0 MB/s 
     |████████████████████████████████| 166 kB 86.1 MB/s 
     |████████████████████████████████| 182 kB 89.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 66.2 MB/s 
     |████████████████████████████████| 162 kB 78.3 MB/s 
     |████████████████████████████████| 162 kB 88.5 MB/s 
     |████████████████████████████████| 158 kB 61.2 MB/s 
     |████████████████████████████████| 157 kB 85.9 MB/s 
     |████████████████████████████████| 157 kB 73.1 MB/s 
     |████████████████████████████████| 157 kB 84.0 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 71.2 MB/s 
     |████████████████████████████████| 157 kB 70.5 MB/s 
     |████████████████████████████████| 156 kB 67.8 MB/s 


In [4]:
!kaggle datasets download -d minjaechoi99/aihub-groom

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [5]:
!unzip /content/aihub-groom.zip

unzip:  cannot find or open /content/aihub-groom.zip, /content/aihub-groom.zip.zip or /content/aihub-groom.zip.ZIP.


In [6]:
!pip install koco

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for koco: filename=koco-0.2.3-py3-none-any.whl size=7362 sha256=cc543855edd5374ab4988f6d5d4c27e36f87ee15acdf9765f974bb5233bbfb1d
  Stored in directory: /root/.cache/pip/wheels/fe/32/23/d694980cffc0b144d06f90951b398172b3e2ceeb6e2c980635
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=6a269d885209d148120e4ae2a1029cc6802440e9091d3a101acf05cab719574c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built koco wget


In [7]:
import torch
import torch.optim as optim

import numpy as np
import tqdm as tqdm
import koco

from transformers import(
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EarlyStoppingCallback,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_constant_schedule,
    AdamW
)

In [8]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## readjson

In [9]:
from pathlib import Path
import json, re
def read_aihub(path):
    path = Path(path)
    with open(path, 'rb') as f:
        print(f)
        squad_dict = json.load(f)
    texts = []
    intensities = []
    for group in squad_dict:
        for passage in group['sentences']:
            text = passage['origin_text']
            text = re.sub('#.*?#', '[UNK]', text)
            intensity = passage['intensity']
            if intensity == 1:
                continue
            texts.append(text)
            if intensity < 1:
                intensity = 0
            else:
                intensity = 1
            intensities.append(intensity)

    return texts ,intensities

In [11]:
import pandas as pd

texts = []
intensities = []
for i in range(1,6):
    file_path =  f'/content/data/Training/talksets-train-{i}.json'
    ttexts , iintensities = read_aihub(file_path)
    texts += ttexts
    intensities += iintensities

hate = 0
train_dev = koco.load_dataset('korean-hate-speech', mode='train_dev')
for line in train_dev['train']:
    texts.append(line['comments'])
    if line['hate'] == 'none':
        intensities.append(0)
    else:
        hate += 1
        intensities.append(1)
for line in train_dev['dev']:
    texts.append(line['comments'])
    if line['hate'] == 'none':
        intensities.append(0)
    else:
        hate += 1
        intensities.append(1)
ai_df = pd.DataFrame(list(zip(texts , intensities)),
            columns =['texts','intensities'])

In [14]:
ai_df

,texts,intensities
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,1
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",1
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1
...,...,...
8362,지현우 범죄 저지르지 않았나요?,1
8363,여자인생 망칠 일 있나 ㅋㅋ,1
8364,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,1
8365,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",1


In [ ]:
ai_df = ai_df.sample(frac=1,random_state=43)

In [ ]:
spliter = int(len(ai_df)*0.8)

In [ ]:
train_df = ai_df.iloc[:spliter,:]
dev_df = ai_df.iloc[spliter:,:]

In [ ]:
dev_df

,texts,intensities
42102,방탕출 게임 하러 왔는데 못 나가겠어.. 갇힌 것 같아,0
77099,음식은 개같이 만들어서 대리점주 전부 망하게 해놓고 지만 잘 먹고 잘사는 샛기.,1
34793,본인 등판했네 ㅉㅉ 걍 발 닦고 잠이나 자라,0
13545,니 얼굴을 보면 알지 딱바도 너 못생겼잖아,1
111097,얘 춤선 장난 아님 일단 보고 까라,0
...,...,...
57651,윗사람들도 다 똥만찼지 뭐.,1
40753,저러니까 지나가는 사람들한테도 손가락질을 당하지.,1
248063,돼지는 뭘 해도 돼지일 뿐이야,1
150848,이 늦은 시간에 자꾸 연락하는 개념 가출한 놈이 있다?,1


In [ ]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_df)
dev_data = Dataset.from_pandas(dev_df)

In [ ]:
train_data

Dataset({
    features: ['texts', 'intensities', '__index_level_0__'],
    num_rows: 236856
})

## tokenize

In [ ]:
model_name = 'beomi/KcELECTRA-base-v2022'
max_length = 128

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def tokenizeWithLabel(data):
    tokenized_datas = tokenizer(
        data['texts'],
        max_length=max_length,
        padding="max_length",
        truncation="only_second"
    )
    tokenized_datas['labels'] = data['intensities']
    return tokenized_datas

In [ ]:
train_tokenized_datasets = train_data.map(tokenizeWithLabel, batched=True, remove_columns=train_data.column_names)
dev_tokenized_datasets = dev_data.map(tokenizeWithLabel, batched=True, remove_columns=dev_data.column_names)

  0%|          | 0/237 [00:00<?, ?ba/s]

  0%|          | 0/60 [00:00<?, ?ba/s]

In [ ]:
train_tokenized_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 236856
})

## train with sweep

In [ ]:
sweep_configuration = {
    'method': 'grid',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'eval/loss'},
    'parameters': 
    {
        'batch_size': {'values': [256]},
        'epochs': {'values': [5]},
        'lr': {'values': [3e-5]},
        'scheduler': {'values': ['linear', 'cosine', 'constant']}
     }
}

In [ ]:
max_batch_size = 256
def train():
    torch.cuda.empty_cache()
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    grouped_params = model.parameters()
    run = wandb.init(config=sweep_configuration, entity="groom2team")
    batch_size = wandb.config.batch_size if wandb.config.batch_size < max_batch_size else max_batch_size
    gradient_accumulation_steps= wandb.config.batch_size // max_batch_size
    epochs = wandb.config.epochs
    total_steps = int(len(train_tokenized_datasets)/wandb.config.batch_size*epochs)
    learning_rate = wandb.config.lr
    data_collator = default_data_collator
    grouped_params = model.parameters()
    optimizer=AdamW(grouped_params, lr=learning_rate)
    scheduler_type = wandb.config.scheduler
    if scheduler_type == 'linear':
        scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=0,
                                                    num_training_steps=total_steps+1)
    elif scheduler_type == 'cosine':
        scheduler=get_cosine_schedule_with_warmup(optimizer=optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps+1)
    elif scheduler_type == 'constant':
        scheduler=get_constant_schedule(optimizer=optimizer)
    optimizers = optimizer, scheduler
    args = TrainingArguments(
        f"{model_name}-finetuned",
        evaluation_strategy = "steps",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        gradient_accumulation_steps = gradient_accumulation_steps,
        report_to="wandb",
        run_name="utopia",
        logging_steps = total_steps//200,
        eval_steps = total_steps//100,
        save_steps = total_steps//100,
        weight_decay=0.0,
        save_total_limit = 2,
        load_best_model_at_end=True
    )
    trainer = Trainer(
        model,
        args,
        train_dataset=train_tokenized_datasets,
        eval_dataset=dev_tokenized_datasets,
        data_collator=data_collator,
        tokenizer=tokenizer,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
        optimizers=optimizers
    )
    trainer.train()# train 하고
    trainer.save_model(output_dir= 'pytorch_finetuned') # trainer에서 실행된 model save
    artifact = wandb.Artifact(name='pytorch_finetuned', type='model') # wandb에 해당 모델 version 관리.
    artifact.add_dir('pytorch_finetuned', name='best_model_at_end')
    run.log_artifact(artifact)

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='pj3_classifier_data_add', entity='groom2team')
count = 3

Create sweep with ID: eacjzlg5
Sweep URL: https://wandb.ai/groom2team/pj3_class_kcelectra/sweeps/eacjzlg5


In [ ]:
wandb.agent(sweep_id, function=train, count=count)

In [ ]:
wandb.finish() # wandb 종료